![header](https://apps.socib.es/socib_logo.png)

# HANDS ON DATA
<div style="text-align: right"><i>DATA ENTITIES: discovery and download</i></div>

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Cell" and select "Run All Above Selected Cell".<br>
***

<h1>Table of Contents<span class="tocSkip"></h1>
<div class="toc">
<ul class="toc-item">
<li>
<span><a href="#1.-Setup" data-toc-modified-id="1.-Setup">1. Setup</a></span>
<ul>
<li><span><a href="#1.1.-Python-packages" data-toc-modified-id="1.1.-Python-packages">1.1. Python packages</a></span></li>
<li><span><a href="#1.2.-Api-key" data-toc-modified-id="1.2.-Api-key">1.2. Api key</a></span></li>
    <li><span><a href="#1.3.-Auxiliary-functions" data-toc-modified-id="1.3.-Auxiliary-functions">1.3. Auxiliary-functions</a></span></li>

</ul>
</li>
<li><span><a href="#2.-Data-entities" data-toc-modified-id="2.-Data-entities">2. Data entities</a></span></li>
<ul>
<li><span><a href="#2.1.-Discovery" data-toc-modified-id="2.1.-Discovery">2.1. Discovery</a></span></li>
<li><span><a href="#2.2.-Donwload" data-toc-modified-id="2.2.-Download">2.2. Download</a></span></li>
</ul>
<li><span><a href="#3.-Wrap-up" data-toc-modified-id="3.-Wrap-up">3. Wrap-up</a></span></li>
</ul>
</div>

***

# 1. Setup

To use [SOCIB API](http://api.socib.es) users will have to perform authenticated http requests. Therefore we need to declare first our api key and also import some libraries in python that facilitates making queries and rendering the response in a friendly way.

## 1.1. Python packages

`Run the next cell`:

In [1]:
import requests
import json
from json2html import *
from IPython.display import HTML
import math
import shutil
import os

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
If any of them raises any error it means you need to install the module first. For doing so please:
1. Open a new cell in the notebook
2. Run <i>`!conda install packageName --yes`</i> or <i>`!pip install packageName`</i>
3. Import again!
<br><br>
Example: <i>how-to-solve import error for json2html module </i>

![errorImporting.gif](./assets/errorImporting.gif)
</div>

## 1.2. Api key

In [2]:
api_key = 'api_dummy' #write you SOCIB api_key

<div class="alert alert-block alert-info">
<b>IMPORTANT</b>
    
***  
If you do not have already an api key, you should request one at [here](http://api.socib.es/home/) before continuing.
</div>

After setting your api key please `run the next cell` to save it on the headers we will use later on:

In [3]:
headers = {
    'accept': 'application/vnd.socib+json',
    'apikey': api_key,
}

## 1.3. Auxiliary functions

Some api responses requires looping in order to see them completely and not in chunks. IN order to better deal with this kind of responses please `run the next cell` to load it into memory for later usage. 

In [4]:
from lib.functions import scbRequestHandler

Let's load into memory also the next function, as it will ease the download of the data from any of the avialble entities. `Run the next cell` 

In [5]:
from lib.functions import scbDownloadHandler

# 2. Data entities

The observations derived from the Data streams managed y SOCIB (see [Getting-started](0q-Getting-started.ipynb) is stored as netCDF files and served in as:

<li>Entries: netCDF files.</li>

<li>Sources: netCDF files grouped in collections based on its sources (instrument/platform reporting the data)</li>

<li>Products netCDF files grouped in collections based on a set of rules (same project, same experiment, same endurance line/site...)</li>

We will use next SOCIB API to know more about these entities in terms of discovery and download.

## 2.1. Discovery

Users can discover the data available at SOCIB as entries, sources or products by using SOCIB API.<br>Le's see how!

Set next the data entity yo wanna explore and `run the cell`:

In [6]:
entity = 'data-products' #options: 'entries', 'data-sources', 'data-products'

Define next some searching citerias and `run the cell`:

In [7]:
searchParams = {
    'platform_type' : 'Research Vessel', 
    'instrument_type' : 'CTD', 
    'initial_datetime' : '2019-01-01T00:00:00', 
    'end_datetime' : '2019-12-31T23:59:59', 
    'standard_variable' : None,
    'bbox' : '39.5,40.5,0.0,1.50', #minlat., maxlat., minlon., maxlon
}

<div class="alert alert-block alert-info">
<b>IMPORTANT</b>
    
***  
If you do not wanna set a specific value on any of the above searching criterias leave it as 'None' remove it from the dictionary.<br>
Example is provided on standard_variable.
</div>

<div class="alert alert-block alert-warning">
<b>WARNING</b>
    
***  
There are more possible searching parameters. You can find all possible ones in the api documentation ([swagger])(http://api.socib.es/). In the above example we have just set the common ones for any of the data entities.
</div>

Let's compose the query parameters!: `run the next cell`

In [8]:
queryParams = '?'
for key,value in searchParams.items():
    if value != None:
        queryParams = queryParams + key+'='+value+'&'
queryParams

'?platform_type=Research Vessel&instrument_type=CTD&initial_datetime=2019-01-01T00:00:00&end_datetime=2019-12-31T23:59:59&bbox=39.5,40.5,0.0,1.50&'

Let's make the formal request!: `run the next cell`

In [9]:
query = 'http://api.socib.es/%s/%s'%(entity,queryParams)
response = scbRequestHandler(query,headers)
print('SOCIB %s catalog comprehends a total of %s elements'%(entity, len(response)))
HTML(json2html.convert(json=response))

SOCIB data-products catalog comprehends a total of 1 elements


## 2.2 Data Download

Data in any entity can be downloaded solely as netCDF files, as this is the format used in the MetOcean community and, therefore, by SOCIB.

<div class="alert alert-block alert-info">
<b>IMPORTANT</b>
    
***  
You can convert to CSV or other file formats from netCDF by relying on python libraries like <b>[xarray](http://xarray.pydata.org/en/stable/)</b>.
</div>

Set next the element form the ebove list whose associate file(s) you wanna download and `run the cell`

In [10]:
element = response[0]#for example the first element found in the above catalog search

Set next the directory you want to download the files to and `run the cell`:

In [11]:
target_dir = os.getcwd()
target_dir

'/home/paz/CODE/socibApiExamples/API_examples'

Let's look for files and download those trough http service:

In [12]:
scbDownloadHandler(element, target_dir, headers)

...Donwloading dep0025_socib-rv_scb-rdi001_L1_2019-05.nc
...Donwloading dep0025_socib-rv_scb-rdi001_L0_2019-05.nc
...Donwloading dep0056_socib-rv_scb-met009_L0_2019-05-07.nc
...Donwloading dep0056_socib-rv_scb-met009_L1_2019-05-07.nc
...Donwloading dep0055_socib-rv_scb-tsl001_L1_2019-05-07.nc
...Donwloading dep0055_socib-rv_scb-tsl001_L0_2019-05-07.nc
...Donwloading dep0014_socib-rv_scb-sbe9001_L0_2019-05-07_data_dt.nc
...Donwloading dep0014_socib-rv_scb-sbe9001_L1_2019-05-07_data_dt.nc
...Donwloading dep0025_socib-rv_scb-rdi001_L1_2019-05.nc
...Donwloading dep0025_socib-rv_scb-rdi001_L0_2019-05.nc
...Donwloading dep0056_socib-rv_scb-met009_L0_2019-05-07.nc
...Donwloading dep0056_socib-rv_scb-met009_L1_2019-05-07.nc
...Donwloading dep0055_socib-rv_scb-tsl001_L1_2019-05-07.nc
...Donwloading dep0055_socib-rv_scb-tsl001_L0_2019-05-07.nc
...Donwloading dep0014_socib-rv_scb-sbe9001_L0_2019-05-07_data_dt.nc
...Donwloading dep0014_socib-rv_scb-sbe9001_L1_2019-05-07_data_dt.nc
...Donwloading d

## 3. Wrap-up

So far, you shoud be familiar with SOCIB available data catalogs (entries, sources and products), as well as the way to discover and download the data in those based on several criterials (temporal coverage, spatial coverage, source instrutment/paltfoms etc).